# Step 4: Generate events at arbitrary benchmark points with MadMiner

n.b.: this notebook is heavily based on the corresponding `MadMiner` one here: https://github.com/madminer-tool/madminer/blob/main/examples/tutorial_particle_physics/3a_likelihood_ratio.ipynb

In [54]:
parameter_code = "c2"

In [55]:
import logging
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

%matplotlib inline

from madminer.sampling import SampleAugmenter
from madminer import sampling

In [56]:
# MadMiner output
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [57]:
import yaml
with open("workflow.yaml", "r") as file:
    workflow = yaml.safe_load(file)

In [58]:
data_input_dir = workflow["sampling"]["input_dir"]
samples_output_dir = workflow["sampling"]["output_dir"]

## Generate signal samples at arbitrary benchmark points

In [59]:
test_split = 0.14

You may want to specify exact benchmark points for test sets. Use the variable `parameter_code` to denote which SMEFT Wilson coefficients you're varying. 

In [60]:
if parameter_code == "c1":
    test_set_codes = {
        "m20": (0, -20, 0),
        "m16": (0, -16, 0),
        "m12": (0, -12, 0),
        "m8": (0, -8, 0),
        "m4": (0, -4, 0),
        "p4": (0, 4, 0),
        "p8": (0, 8, 0),
        "p12": (0, 12, 0),
        "p16": (0, 16, 0),
    }
elif parameter_code == "c0":
    test_set_codes = {
        "m12": (-12, 0, 0),
        "m10": (-10, 0, 0),
        "m8": (-8, 0, 0),
        "m6": (-6, 0, 0),
        "m4": (-4, 0, 0),
        "m2": (-2, 0, 0),
        "p2": (2, 0, 0),
        "p1": (1, 0, 0),
    }
elif parameter_code == "c0c1":
    test_set_codes = {
        "m10p2p0": (-10, 2, 0),
        "p3m2p0": (3, -2, 0),
        "m4p1p0": (-4, 1, 0),
    }
elif parameter_code == "c0c2":
    test_set_codes = {
        "m10p0p3": (-10, 0, 3),
        "p3p0m2": (3, 0, -2),
        "m4p0p3": (-4, 0, 3),
    }
elif parameter_code == "c1c2":
    test_set_codes = {
        "p0m2p2": (0, -2, 2),
        "p0m3p1": (0, -3, 1),
        "p0m1p3": (0, -1, 3),
    }
else:
    raise ValueError(f"Unknown parameter_code: {parameter_code}")


print(list(test_set_codes.keys()))

printed_codes = []
for c in test_set_codes.keys():
    printed_codes.append([test_set_codes[c][0]/10.0,test_set_codes[c][1]/10.0,test_set_codes[c][2]/10.0])

print(printed_codes)


ValueError: Unknown parameter_code: c2

# Signal Events

Note that the line `theta=sampling.random_morphing_points(1000, [("flat", -14, 6), ("flat", -4, 5), ("flat", -5, 7)]),` will have to be modified if you only want to scan over 1 Wilson coefficient.

In [ ]:
sampler = SampleAugmenter(f'{data_input_dir}/delphes_s_shuffled_100TeV.h5')


# alternative training set
x, theta, n_effective = sampler.sample_train_plain(
    theta=sampling.random_morphing_points(1000, [("flat", -14, 6), ("flat", -4, 5), ("flat", -5, 7)]),
    n_samples=10000000,
    folder=f'{samples_output_dir}/plain_real/delphes_s' + f"/{parameter_code}",
    filename=f"alt_{parameter_code}",
    sample_only_from_closest_benchmark=True,
    n_processes=16,
    validation_split = 0.0,
    test_split = test_split
    )

# alternative test sets
for code in test_set_codes.keys():

    _ = sampler.sample_test(
        theta=sampling.morphing_point(test_set_codes[code]),
        n_samples=10000,
        folder=f'{samples_output_dir}/plain_real/delphes_s' + f"/{parameter_code}",
        filename=f"alt_{parameter_code}_{code}_test",
        sample_only_from_closest_benchmark=True,
        validation_split = 0.0,
        test_split = test_split
        )


# SM training set
x, theta, n_effective = sampler.sample_train_plain(
    theta=sampling.benchmark("sm"),
    n_samples=10000000,
    folder=f'{samples_output_dir}/plain_real/delphes_s' + f"/{parameter_code}",
    filename="sm",
    sample_only_from_closest_benchmark=True,
    n_processes=1,
    validation_split = 0.0,
    test_split = test_split
    )

# SM test set
_ = sampler.sample_test(
    theta=sampling.benchmark("sm"),
    n_samples=100000,
    folder=f'{samples_output_dir}/plain_real/delphes_s' + f"/{parameter_code}",
    filename=f"sm_test",
    sample_only_from_closest_benchmark=True,
    validation_split = 0.0,
    test_split = test_split
    )

14:09 madminer.analysis.da INFO    Loading data from /vols/cms/us322/post_delphes_data/delphes_s_shuffled_100TeV.h5
14:09 madminer.utils.inter INFO    HDF5 file does not contain nuisance parameters information
14:09 madminer.utils.inter INFO    HDF5 file does not contain finite difference information
14:09 madminer.utils.inter INFO    HDF5 file does not contain systematic information
14:09 madminer.analysis.da INFO    Found 3 parameters
14:09 madminer.analysis.da INFO      0: cp (LHA: DIM6 5, Power: 2, Range: (-16, 8))
14:09 madminer.analysis.da INFO      1: cdp (LHA: DIM6 4, Power: 2, Range: (-5, 6))
14:09 madminer.analysis.da INFO      2: ctp (LHA: DIM62F 19, Power: 2, Range: (-6, 8))
14:09 madminer.analysis.da INFO    Did not find nuisance parameters
14:09 madminer.analysis.da INFO    Found 10 benchmarks
14:09 madminer.analysis.da INFO    Found 20 observables
14:09 madminer.analysis.da INFO    Found 9456 events
14:09 madminer.analysis.da INFO      1034 signal events sampled from ben

## Generate background samples

In [ ]:
sampler = SampleAugmenter(f'{data_input_dir}/delphes_b0_shuffled_100TeV.h5')


# bkg training set
x, theta, n_effective = sampler.sample_train_plain(
    theta=sampling.benchmark("sm"),
    n_samples=10000000,
    folder=f'{samples_output_dir}/plain_real/delphes_b0' + f"/{parameter_code}",
    filename="bkg",
    sample_only_from_closest_benchmark=True,
    n_processes=1,
    validation_split = 0.0,
    test_split = test_split
    )


# bkg test set
_ = sampler.sample_test(
    theta=sampling.benchmark("sm"),
    n_samples=100000,
    folder=f'{samples_output_dir}/plain_real/delphes_b0' + f"/{parameter_code}",
    filename=f"bkg_test",
    sample_only_from_closest_benchmark=True,
    validation_split = 0.0,
    test_split = test_split
    )


14:11 madminer.analysis.da INFO    Loading data from /vols/cms/us322/post_delphes_data/delphes_b0_shuffled_100TeV.h5


14:11 madminer.utils.inter INFO    HDF5 file does not contain nuisance parameters information
14:11 madminer.utils.inter INFO    HDF5 file does not contain finite difference information
14:11 madminer.utils.inter INFO    HDF5 file does not contain systematic information
14:11 madminer.analysis.da INFO    Found 3 parameters
14:11 madminer.analysis.da INFO      0: cp (LHA: DIM6 5, Power: 2, Range: (-16, 8))
14:11 madminer.analysis.da INFO      1: cdp (LHA: DIM6 4, Power: 2, Range: (-5, 6))
14:11 madminer.analysis.da INFO      2: ctp (LHA: DIM62F 19, Power: 2, Range: (-6, 8))
14:11 madminer.analysis.da INFO    Did not find nuisance parameters
14:11 madminer.analysis.da INFO    Found 10 benchmarks
14:11 madminer.analysis.da INFO    Found 20 observables
14:11 madminer.analysis.da INFO    Found 1558 events
14:11 madminer.analysis.da INFO      0 signal events sampled from benchmark sm
14:11 madminer.analysis.da INFO      0 signal events sampled from benchmark morphing_basis_vector_1
14:11 mad